In [27]:
import pdblp
from xbbg import blp
import datetime
import workdays
import pandas as pd
import numpy as np
from statistics import mean

term = "1M"
C = ["EUR","GBP","USD","CAD","AUD","NZD","CHF","NOK","SEK","CNH"]
T = [ x + "JPY" + term + " BGN Curncy" for x in C ]

d_from = "20210401"
d_to = workdays.workday(datetime.datetime.today(), days=-1).strftime("%Y%m%d")

F = ["px_ask","px_bid","px_last"]
bdh = blp.bdh(T, F, d_from, d_to, Calendar="5D", Fill="P", FWD_CURVE_QUOTE_FORMAT="OUTRIGHT" ).reset_index()

S = pd.DataFrame(np.zeros([len(bdh),1]), columns = ["Date"]) #Spread
S["Date"] = bdh["index"]


tmp = ["ask","bid","last"]
for i in C:
    for j in tmp: 
        S[ i + "_" + j ] = bdh[i + "JPY" + term + " BGN Curncy"]["px_" + j].values
        
    S[i + "_Spread"] = S[i + "_ask"] - S[i + "_bid"]
    S[i + "_Cost"] = S[i + "_Spread"] / S[i + "_last"]

S

,Date,EUR_ask,EUR_bid,EUR_last,EUR_Spread,EUR_Cost,GBP_ask,GBP_bid,GBP_last,GBP_Spread,...,SEK_ask,SEK_bid,SEK_last,SEK_Spread,SEK_Cost,CNH_ask,CNH_bid,CNH_last,CNH_Spread,CNH_Cost
0,2021-04-01,130.3549,130.2714,130.3131,0.0835,0.000641,153.0117,152.9135,152.9631,0.0982,...,12.6985,12.6808,12.6896,0.0177,0.001395,16.800797,16.760542,16.780669,0.040255,0.002399
1,2021-04-02,130.5109,129.9174,130.2091,0.5935,0.004558,153.3834,152.8549,153.1196,0.5285,...,12.7083,12.6575,12.6829,0.0508,0.004005,16.850576,16.730297,16.790436,0.120279,0.007164
2,2021-04-05,130.2274,130.1533,130.1853,0.0741,0.000569,153.2101,153.0328,153.1209,0.1773,...,12.6653,12.6524,12.6589,0.0129,0.001019,16.790999,16.728172,16.759585,0.062827,0.003749
3,2021-04-06,130.4167,130.3236,130.3652,0.0931,0.000714,151.7345,151.6282,151.6814,0.1063,...,12.7506,12.7279,12.7392,0.0227,0.001782,16.777531,16.688047,16.732789,0.089484,0.005348
4,2021-04-07,130.4646,130.4014,130.4330,0.0632,0.000485,150.9163,150.8216,150.8695,0.0947,...,12.7468,12.7263,12.7366,0.0205,0.001610,16.775402,16.698183,16.736792,0.077219,0.004614
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
642,2023-09-18,157.2949,157.2105,157.2527,0.0844,0.000537,181.9462,181.8472,181.8967,0.0990,...,13.2104,13.1892,13.1998,0.0212,0.001606,20.159418,20.146357,20.152937,0.013061,0.000648
643,2023-09-19,157.3338,157.2490,157.2914,0.0848,0.000539,182.3110,182.2470,182.2785,0.0640,...,13.1985,13.1811,13.1898,0.0174,0.001319,20.157860,20.145569,20.151765,0.012291,0.000610
644,2023-09-20,157.5993,157.5201,157.5647,0.0792,0.000503,182.2500,182.1779,182.2140,0.0721,...,13.2696,13.2588,13.2642,0.0108,0.000814,20.224026,20.211424,20.217775,0.012602,0.000623
645,2023-09-21,156.8765,156.7416,156.8140,0.1349,0.000860,180.7162,180.5597,180.6384,0.1565,...,13.1673,13.1558,13.1616,0.0115,0.000874,20.104518,20.091946,20.098232,0.012572,0.000626


In [29]:
F = ["Spread","Cost","last"]
SC = pd.DataFrame(np.zeros([len(C),len(F)]), index = C, columns = F)

for i in C:
    for j in F:
        SC.loc[i,j] = mean(S[ i + "_" + j])

tmp = ["Spread","last"]
for i in tmp: 
    SC[i] = SC[i].apply("{:.2f}".format)
SC["Cost"] = SC["Cost"].apply("{:.2%}".format)

print("vsJPY Both Ways")
SC

vsJPY Both Ways


,Spread,Cost,last
EUR,0.23,0.16%,139.01
GBP,0.27,0.17%,161.68
USD,0.10,0.08%,126.88
CAD,0.22,0.23%,97.53
AUD,0.07,0.08%,88.72
NZD,0.21,0.25%,82.07
CHF,0.31,0.22%,137.09
NOK,0.02,0.18%,13.26
SEK,0.02,0.17%,12.94
CNH,0.04,0.22%,18.79


In [19]:
# https://www.sbifxt.co.jp/service/detailedspread.html
SC_sbi = pd.DataFrame(np.zeros([len(C),len(F)]), index = C, columns = F)
SC_sbi["last"] = SC["last"].values

tmp = "Spread"

SC_sbi.loc["USD",tmp] = 0.4 / 100
SC_sbi.loc["EUR",tmp] = 0.5 / 100
SC_sbi.loc["GBP",tmp] = 0.9 / 100
SC_sbi.loc["AUD",tmp] = 0.6 / 100
SC_sbi.loc["NZD",tmp] = 1.2 / 100
SC_sbi.loc["CAD",tmp] = 2.5 / 100
SC_sbi.loc["CHF",tmp] = 2.8 / 100
SC_sbi.loc["SEK",tmp] = 2.8 / 100
SC_sbi.loc["NOK",tmp] = 3.5 / 100
SC_sbi.loc["CNH",tmp] = 2.5 / 100

for i in C:
    SC_sbi.loc[i,"Cost"] = SC_sbi.loc[i,"Spread"] / float(SC_sbi.loc[i,"last"])

SC_sbi["Cost"] = SC_sbi["Cost"].apply("{:.3%}".format)

print("vsJPY Both Ways")
SC_sbi

vsJPY BothWays


,Spread,Cost,last
EUR,0.005,-0.040%,-12.59
GBP,0.009,-0.028%,-32.00
USD,0.004,-0.013%,-30.17
CAD,0.025,-0.117%,-21.41
AUD,0.006,-0.040%,-15.04
NZD,0.012,-0.061%,-19.62
CHF,0.028,-0.976%,-2.87
NOK,0.035,-1.542%,-2.27
SEK,0.028,-1.854%,-1.51
CNH,0.025,-0.552%,-4.53
